In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime
import ipywidgets as widgets


import yfinance as yf
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

import sambo
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy

from backtesting.test import SMA, GOOG

# backtesting.set_bokeh_output(notebook=True)
            
import itertools 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True

e:\Quant Trading\trader-joe\.venv\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
from utils.loader import *
from utils.signals import *
from utils.trade import *
from utils.strategy import *

In [3]:
# now = datetime.today().strftime('%Y-%m-%d')

loader = DataLoader(ticker='AAPL', start='2010-01-01', end='2024-12-31', freq='1d', test_size=0.2)
loader.run()
# test     

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [4]:
def OBV(price: pd.Series, volume: pd.Series) -> pd.Series:
    # Calculate the difference in price and fill the first NaN with 0
    diff = price.diff().fillna(0)
    
    # Use numpy's vectorized sign function to get the sign of each change
    sign = np.sign(diff)
    
    # Multiply the sign by the volume and compute the cumulative sum to get OBV
    obv = (sign * volume).cumsum()
    
    return obv

class EMACross_test(Strategy):

    short_duration = 5  # Default values, can be overridden
    long_duration = 10
    atr_period = 14
    id = 0
    take_profit_ratio = 1.3
    take_profit_next_order = 1.10
    stop_loss_ratio = 0.8

    stop_loss_duration = 3

    lookback = 3
    atr_multiplier = 1

    def init(self):
        price = self.data.Close
        volume = self.data.Volume
        data = self.data
        self.ma1 = self.I(EMA, price, self.short_duration, plot=True, overlay=True)
        self.ma2 = self.I(EMA, price, self.long_duration, plot=True, overlay=True)
        self.bbw = self.I(BBW, price, plot=False, overlay=False)
        # self.atr = self.I(ATR, price, plot=True, overlay=False)
        #self.obv = self.I(OBV, self.data.Close, self.data.Volume)

        self.atr = self.I(ATR, self.data, self.atr_period)
        self.previous_low = self.I(previous_low, self.data.Low, self.stop_loss_duration, plot = False)
        self.previous_high = self.I(previous_high, self.data.High, self.stop_loss_duration, plot = False)

    def next(self):
        entry_price = self.data.Close[-1]

        long_stop_loss = min(self.previous_low[-1], entry_price - self.atr_multiplier*self.atr[-1])
        long_stop_loss_next = min(self.previous_low[-1], entry_price - self.atr_multiplier*(1/2)*self.atr[-1])

        for trade in self.trades:
            if trade.tag == "Long":
                trade.sl = long_stop_loss
            if trade.tag == "Next":
                trade.sl = long_stop_loss_next


        past_max = max(self.bbw[-self.lookback-1:-1])  # Maximum in last 4 periods
        current_bbw = self.bbw[-1]

        count_green = sum(self.data.Color[-2:])
        
        #add if current OBV > average OBV -> buy
        if crossover(self.ma1, self.ma2) and count_green >= 1:
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Long {self.id}')
            self.id += 1
        
        #for this kind of order, set take_profit smaller (mua duoi), sl tighter
        elif self.ma1[-1] > self.ma2[-1] and self.ma1[-2] > self.ma2[-2] and count_green >= 2 and not self.position:
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_next_order, sl=entry_price*self.stop_loss_ratio, tag=f'Next')
            self.id += 1
            
        elif crossover(self.ma2, self.ma1):
            for trade in self.trades:
                trade.close()

        # elif crossover(self.ma2, self.ma1):
        #     for trade in self.trades:
        #         if trade.tag ==f'Long {self.id-1}':
        #             trade.close()

strategy = EMACross_test
# strategy = BollingerBound
bt = BackTrader(data=loader.data)

# params = {
#     'short_duration': 5,
#     'long_duration': 10
# }
params = {'take_profit_ratio': 1.3}
bt.evaluate(data=bt.train_data, strategy=strategy, params=params, order_size=0.9999, plot=True)
# bt.trades.head()

Start                     2010-01-04 00:00:00
End                       2021-12-28 00:00:00
Duration                   4376 days 00:00:00
Exposure Time [%]                    64.54606
Equity Final [$]                  115945.6528
Equity Peak [$]                  116287.55015
Commissions [$]                   14499.89519
Return [%]                         1059.45653
Buy & Hold Return [%]               2860.9543
Return (Ann.) [%]                    22.70555
Volatility (Ann.) [%]                22.78889
CAGR [%]                             15.15613
Sharpe Ratio                          0.99634
Sortino Ratio                         1.88128
Calmar Ratio                          0.83017
Max. Drawdown [%]                   -27.35044
Avg. Drawdown [%]                    -3.57997
Max. Drawdown Duration      790 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                  123
Win Rate [%]                         45.52846
Best Trade [%]                    

In [5]:
# grid_search = {
#     'short_duration': range(2, 4),
#     'long_duration': range(5, 11)
# }
grid_search={'take_profit_ratio': [1.3], 'stop_loss_ratio': [0.9]}

bt.cross_val(strategy=strategy, train_size=240, test_size=240, step_size=240, order_size=0.999, commission=0.002, grid=grid_search)

{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.float64(0.9)}
7627.163589426518
{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.float64(0.9)}
15390.714719265228
{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.float64(0.9)}
10636.572424567788
{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.float64(0.9)}
10074.917840732132
{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.float64(0.9)}
10551.038789670201
{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.float64(0.9)}
11598.681619104531
{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.float64(0.9)}
13891.664553499197
{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.float64(0.9)}
11978.815575608874
{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.float64(0.9)}
12259.815152137598
{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.float64(0.9)}
15636.384676357946
{'take_profit_ratio': np.float64(1.3), 'stop_loss_ratio': np.

11987.443540913646

In [6]:
for point in train:
    if point < 0:
        print(point)
        
    print(point)

NameError: name 'train' is not defined